# Stacking

## Data Cleaning

In [53]:
import pandas as pd
datapath = 'data/'
fname = 'cs-training.csv'
data = pd.read_csv(datapath + fname, index_col=0)

# data_sto = data.sample(frac=1)
data_sto = data
Y_sto = data_sto['SeriousDlqin2yrs']
X_sto = data_sto.drop('SeriousDlqin2yrs', axis=1)
X_sto['MissingIncome'] = X_sto['MonthlyIncome'].isnull().apply(lambda b: int(b))

def replace_nan_c(x, c):
    if pd.isnull(x):
        return c
    else:
        return x

X_sto['MonthlyIncome'] = X_sto['MonthlyIncome'].apply(lambda x: replace_nan_c(x, -1))
X_sto['NumberOfDependents'] = X_sto['NumberOfDependents'].apply(lambda x: replace_nan_c(x, 0))

Xtest = pd.read_csv(datapath + 'cs-test.csv', index_col=0).drop('SeriousDlqin2yrs', axis=1)
Xtest['MissingIncome'] = Xtest['MonthlyIncome'].isnull().apply(lambda b: int(b))
Xtest['MonthlyIncome'] = Xtest['MonthlyIncome'].apply(lambda x: replace_nan_c(x, -1))
Xtest['NumberOfDependents'] = Xtest['NumberOfDependents'].apply(lambda x: replace_nan_c(x, 0))

### Normal Random Forest

In [54]:
Xtest = pd.read_csv(datapath + 'cs-test.csv', index_col=0).drop('SeriousDlqin2yrs', axis=1)
Xtest['MissingIncome'] = Xtest['MonthlyIncome'].isnull().apply(lambda b: int(b))
Xtest['MonthlyIncome'] = Xtest['MonthlyIncome'].apply(lambda x: replace_nan_c(x, -1))
Xtest['NumberOfDependents'] = Xtest['NumberOfDependents'].apply(lambda x: replace_nan_c(x, 0))

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_sto, Y_sto)

prediction = clf.predict_proba(Xtest)
Xtest['Probability'] = prediction[:, 1]
Xtest['Id'] = Xtest.index
submission1 =Xtest[['Id', 'Probability']]

submission1.index.name = 'Id'
submission1 = submission1.drop('Id', axis=1)
submission1.to_csv('submission1.csv')

### Adaboost

In [55]:
Xtest = pd.read_csv(datapath + 'cs-test.csv', index_col=0).drop('SeriousDlqin2yrs', axis=1)
Xtest['MissingIncome'] = Xtest['MonthlyIncome'].isnull().apply(lambda b: int(b))
Xtest['MonthlyIncome'] = Xtest['MonthlyIncome'].apply(lambda x: replace_nan_c(x, -1))
Xtest['NumberOfDependents'] = Xtest['NumberOfDependents'].apply(lambda x: replace_nan_c(x, 0))

from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100)
clf.fit(X_sto.as_matrix(), Y_sto.as_matrix())

prediction = clf.predict_proba(Xtest.as_matrix())
Xtest['Probability'] = prediction[:, 1]
Xtest['Id'] = Xtest.index
submission2 =Xtest[['Id', 'Probability']]

submission2.index.name = 'Id'
submission2 = submission2.drop('Id', axis=1)
submission2.to_csv('submission2.csv')

## Stacking 

In [60]:
submission = submission1.rename(columns={'Probability': 'Prediction1'})
submission['Prediction2'] = submission2['Probability']

In [62]:
submission.head(5)

,Prediction1,Prediction2
Id,,
1,0.076958,0.493671
2,0.034889,0.493673
3,0.034889,0.489306
4,0.063903,0.493587
5,0.076958,0.494360


In [64]:
Xtest = pd.read_csv(datapath + 'cs-test.csv', index_col=0).drop('SeriousDlqin2yrs', axis=1)
Xtest['MissingIncome'] = Xtest['MonthlyIncome'].isnull().apply(lambda b: int(b))
Xtest['MonthlyIncome'] = Xtest['MonthlyIncome'].apply(lambda x: replace_nan_c(x, -1))
Xtest['NumberOfDependents'] = Xtest['NumberOfDependents'].apply(lambda x: replace_nan_c(x, 0))

from xgboost.sklearn import XGBClassifier  
from xgboost.sklearn import XGBRegressor
grid = {'max_depth':10}
xclas =  XGBClassifier()
xclas.set_params(**grid)
xclas = XGBClassifier()  
xclas.fit(X_sto, Y_sto) 
prediction = xclas.predict_proba(Xtest)

Xtest['Probability'] = prediction[:, 1]
Xtest['Id'] = Xtest.index
submission_stacking =Xtest[['Id', 'Probability']]

submission_stacking.index.name = 'Id'
submission_stacking = submission_stacking.drop('Id', axis=1)
submission_stacking.to_csv('submission_stacking.csv')

In [65]:
submission_stacking

,Probability
Id,
1,0.080186
2,0.044407
3,0.013913
4,0.077355
5,0.102172
6,0.030543
7,0.046119
8,0.031583
9,0.005005
